In [1]:
import os
wd = os.getcwd()
wd = os.path.dirname(os.path.dirname(wd))
import sys
sys.path.append(wd+"\\src\\")

In [2]:
from tube import InterpXY,Tube,BorderSimple
from math import *
import numpy as np
from powderlayer import PowderBD, PowderOvLayer, Powder
from gaslayer import GasEOS, GasLayer
from pistonlayer import ElPistEOS, ElPistLayer
import pprint as pp

In [3]:
from tqdm import tqdm_notebook
from matplotlib.patches import Polygon
import matplotlib as mpl
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed
import ipywidgets
mpl.rcParams['figure.figsize'] = (5,3) 

import json
from functional import seq

In [4]:
with open(wd+"\\src\\" + 'gpowders.json') as f:
    all_powders = json.load(f)
all_powders.keys()

dict_keys(['СФ 033', 'ВУ фл', 'ВТ', 'ВТМ', 'ВТХ-10', 'ВТХ-20', 'ВТД-25', '4\\1', '4\\1 фл', '4\\7', '4\\7 св', '4\\7 фл ВБП', '4\\7 ПТ-3 фл', '4\\7 Ц гр', '5\\1', '5\\1 х-10', '5\\1 х-20', '5\\1 д-20', '5\\1 д-25', '5\\7 св', '5\\7 н\\а', '5\\7 в\\а', '5\\7 Ц фл', '6\\7 гр', '6\\7 БП гр', '6\\7 фл В БП', '6\\7 П-5 БП фл', '6\\7 фл ', '7\\1', '7\\1 фл БП', '7\\7', '7\\14', '7\\1 УГ', '8\\1 тр', '8\\7', '8\\1 УГ', '9\\7', '9\\7 МН', '9\\7 БП', '11\\1 БП', '11\\7', '11\\1 УГ', '12\\1 Тр', '12\\1 тр БП', '12\\1 тр МН', '12\\7', '12\\7 В\\А', '12\\1 УГ', '14\\1 тр В\\А', '14\\7', '14\\7 В\\А', '14\\7 БП', '15\\1 тр В\\А', '15\\7', '16\\1 тр', '16\\1 тр В\\А', '17\\7', '18\\1 тр', '18\\1 тр БП', '22\\1 тр', '22\\7', '23\\7', '28\\1 тр', '37\\1 тр', 'НДТ-3 16\\1', 'НДТ-3 18\\1', 'НДТ-3 19\\1', 'НДТ-3 23\\1', 'НДТ-3 32\\1', 'НДТ-3 152\\57', 'ДГ-2 15\\1', 'ДГ-2 17\\1', 'ДГ-3 13\\1', 'ДГ-3 14\\1', 'ДГ-3 17\\1', 'ДГ-3 18\\1', 'ДГ-3 20\\1', 'ДГ-3 23\\1', 'ДГ-4 13\\1', '100\\56', '100\\70', '130\\5

In [5]:
tps = sorted([(I_k, k) for k, d in all_powders.items() if isinstance(I_k:=d['I_k'], float) and I_k > 0.1])
good_marks = [name for I_k, name in tps]
I_ks = [I_k for I_k, name in tps]

In [6]:
tps

[(0.15, 'ВУ фл'),
 (0.17, 'СФ 033'),
 (0.22, 'ВТ'),
 (0.23, 'ВТМ'),
 (0.25, '5\\1'),
 (0.27, 'ВТХ-10'),
 (0.28, '4\\7 ПТ-3 фл'),
 (0.3, '4\\1'),
 (0.3, '4\\7 Ц гр'),
 (0.32, '4\\7'),
 (0.32, '5\\7 н\\а'),
 (0.33, '4\\7 св'),
 (0.35, '5\\7 Ц фл'),
 (0.35, '5\\7 св'),
 (0.37, '5\\7 в\\а'),
 (0.38, '4\\7 фл ВБП'),
 (0.39, 'ВТХ-20'),
 (0.4, '4\\1 фл'),
 (0.4, '7\\14'),
 (0.43, '6\\7 фл '),
 (0.43, '6\\7 фл В БП'),
 (0.45, 'ВТД-25'),
 (0.48, '6\\7 П-5 БП фл'),
 (0.5, '5\\1 х-10'),
 (0.5, '6\\7 гр'),
 (0.5, '7\\1'),
 (0.5, '7\\7'),
 (0.5, '8\\7'),
 (0.51, '7\\1 фл БП'),
 (0.56, '6\\7 БП гр'),
 (0.6, '9\\7'),
 (0.68, '11\\7'),
 (0.68, '5\\1 д-20'),
 (0.7, '5\\1 х-20'),
 (0.7, '8\\1 тр'),
 (0.72, '12\\7 В\\А'),
 (0.76, '9\\7 МН'),
 (0.81, '12\\7'),
 (0.82, '5\\1 д-25'),
 (0.83, '14\\7 В\\А'),
 (0.84, '14\\1 тр В\\А'),
 (0.87, '12\\1 Тр'),
 (0.88, '12\\1 тр МН'),
 (0.9, '7\\1 УГ'),
 (0.9, 'ДГ-4 13\\1'),
 (0.91, '15\\1 тр В\\А'),
 (0.93, '15\\7'),
 (0.94, '9\\7 БП'),
 (0.96, '14\\7'),
 (1.0, '8\

In [7]:
from opti.Chromosome import DRange, SRange,ChromoController
from zone import Zone

In [8]:
L1 = DRange(2,6,'L1')
L2 = DRange(1,4,'L2')
Lcone = DRange(0.3,2.5,'Lcone')


d2 = DRange(0.085,0.125, 'd2') 
chr_contr = ChromoController([L1, L2, Lcone, d2],)


In [9]:
chr_contr.get_chromo()

{'L1': 5.3221650452268925,
 'L2': 1.372529120594666,
 'Lcone': 2.3876247550094125,
 'd2': 0.11922210267809327}

In [10]:
d = {
    'id': 'some_id',
    'tube_points': [[0.0, 0.125], [5.0, 0.125]],
    'layers': [
        {
            'type': 'gas',
            'x_1': 0,
            
        }
    ],
    'borders': []
    
}

In [11]:
def forward_foo(d):
    t = Tube.get_standart({'tube_points': d['tube_points']})
    print(t)
    
    return {'id': d['id']}
forward_foo(d)

Tube(xs=[0.0, 5.0], ds=[0.125, 0.125])


{'id': 'some_id'}

In [18]:
def foo_tst(sandwich_dict_sample):
    from tube import InterpXY,Tube,BorderSimple
    import numpy as np
    from gaslayer import GasEOS, GasLayer
    from powderlayer import PowderOvLayer
    from pistonlayer import ElPistLayer
    
    class Sandwich(object):
        @classmethod
        def get_standart(cls, sandwich_dict):
            tube = Tube.get_standart(sandwich_dict['tube'])
            res = cls(tube)
            x_left = sandwich_dict.get('x_left', 0)
            b = BorderSimple.get_standart(sandwich_dict['borders'][0])
            res.borders.append(b)
            for lr_dict, b_right_dict in zip(sandwich_dict['layers'], sandwich_dict['borders'][1:]):
                if lr_dict['type'] == 'gas':
                    lr = GasLayer.get_standart(tube, x_left, lr_dict, sandwich_dict['calc_settings']['gas_settings'])
                elif lr_dict['type'] == 'powder':
                    lr = PowderOvLayer.get_standart(tube, x_left, lr_dict, sandwich_dict['calc_settings']['powder_settings'])
                elif lr_dict['type'] == 'pist':
                    lr = ElPistLayer.get_standart(tube, x_left, lr_dict, sandwich_dict['calc_settings']['pist_settings'])
                else:
                    t = lr_dict["type"]
                    raise AttributeError(f'Неправильный тип области {t}')
                b = BorderSimple.get_standart(b_right_dict)
                res.layers.append(lr)
                res.borders.append(b)
                x_left = lr.xs_borders[-1] + b.w_border
            res.connect_borders_layers()
            return res

        def __init__(self, tube: Tube, i=0):
            self.tube = tube
            self.i = i
            self.layers = []
            self.borders= []

        def connect_borders_layers(self):
            if len(self.layers)+1 != len(self.borders):
                raise AttributeError(f'Неправильное количество границ и слоев len(self.layers)={len(self.layers)}  len(self.borders)={len(self.borders)}')
            all_layers = [None] + self.layers + [None]
            for b, lr_left, lr_right in zip(self.borders, all_layers[:-1], all_layers[1:]):
                b.lr_left = lr_left
                b.lr_right = lr_right

        def simple_step(self, courant):
            tau = min([lr.get_tau_min() for lr in self.layers]) * courant
            for b in self.borders:
                b.update_xs()
            for b in self.borders:
                b.update_Vs(tau)
            result = Sandwich(self.tube, self.i+1)
            result.layers = [lr.step_up(tau) for lr in self.layers]
            result.borders= [b.copy() for b in self.borders]
            result.connect_borders_layers()
            return result

        def plot(self, fig, ax, **kwargs):
            self.tube.plot(fig, ax, **kwargs)
            for lr in self.layers:
                lr.plot(fig, ax, **kwargs)
            for b in self.borders:
                b.plot(fig, ax, **kwargs)

        def plot_param(self, fig, ax, param_name, **kwargs):
            for lr in self.layers:
                xs = np.array(lr.xs_cells)
                ys = np.array(getattr(lr, param_name))
                color = lr.color_4_plot
                lw = kwargs.get('lw', 2)
                ax.plot(xs, ys, color=color, lw=lw)

        def get_t(self):
            ts = [lr.time for lr in self.layers]
            if abs(min(ts) - max(ts)) > 1e-14:
                print(f'Слои не синхронизированны {ts}')
            return ts[0]

        def get_vel(self):
            return self.layers[-1].Vs_borders[-1]

        def get_x_projectile(self):
            return self.layers[-1].xs_borders[-1]


        def get_p_max(self):
            return max([np.max(lr.ps) for lr in self.layers])

        def __str__(self):
            res = f'Sandwich i={self.i}; t={self.get_t()*1000} ms;  vel={self.get_vel()} m/s;  p_max={self.get_p_max()*1e-6} MPa\nlayers:\n'
            for i, lr in enumerate(self.layers):
                res += f'{i}) {lr}\n'
            return res

        def __repr__(self):
            return str(self)
    try:
        sandw = Sandwich.get_standart(sandwich_dict_sample)
        for i in range(1500):
            if sandw.get_x_projectile() > sandw.tube.get_x_right():
                break
            v = sandw.layers[-1].Vs_borders[-1]
            sandw = sandw.simple_step(np.random.uniform(0.2,0.49))

        return sandw.layers[-1].time
    except Exception as e:
        return str(e)


In [19]:
sandwich_dict_sample = {
    'tube': {
#         'tube_points': [[-1.3, 0.8], [-0.32, 0.8], [-0.28, 0.265],[6.5, 0.265]]
        'tube_points': [[-9.413427684822523, 0.125],[6.2, 0.125]]
    },
    'x_left': -9.413427684822523,
    'layers': [
         {
            'type': 'gas',
            'name': 'N_2',
            'gamma': 1.4,
            'kappa': 0, #0.0010838,
            'R': 297,
            'T_0': 283, # K
            'W_0': 0.00003,  # м^3 объем газа
            'p_0': 10e6, # начальное давление газа
            'u_0': 0,     #начальная скорость
            'x_2': 0
        }, {
            'type': 'pist',
            'omega': 3.03,  # кг
            'p_0': 5e6, # начальное давление в полиэтилене
            'u_0': 0,     #начальная скорость
            'ElPistEOS_kwargs': dict(
                k=1.63098, 
                c_0=2308, 
                ro_0=919.03, 
                sigma_star=25.2, 
                k_0=0.054, 
                b_1=0.027, 
                b_2=0.00675, 
                tau_0=1.36, 
                mu=0.001, 
                tau_s=1,                 
                zeroP=False, 
                zeroE=True)
        },
        {
            'type': 'powder',
            'powder': {
                'name': '4\\7',
                'f': 1.027,
                'etta': 0.228,
                'alpha_k': 1.008,
                'T_1': 3006.0,
                'ro': 1.6,
                'I_k': 0.32,
                'Z_k': 1.488,
                'k_1': 0.811,
                'lambda_1': 0.081,
                'k_2': 0.505,
                'lambda_2': -1.024,
                'k_f': 0.0003,
                'k_l': 0.0016
            },
            'omega': 10035,  # грамм
            'delta': 700, # г/cm^3
            'p_0': 5e6, # начальное давление
            't_ign': 0.002, # начало горения
            'u_0': 0,     #начальная скорость
        }
        
    ],
    'borders': [
        {
            'mass': 1000, # kg
            'p_f': 100000e6, #Pa
            't_init': -1,
            'w_border': 0.05, #m
        },
        {
            'mass': 0.1, # kg
            'p_f': 1, #Pa
            't_init': -1,
            'w_border': 0, #m
#             'p_a_right': 1e5
        },
        {
            'mass': 0.9, # kg
            'p_f': 1, #Pa
            't_init': -1,
            'w_border': 0, #m
#             'p_a_right': 1e5
        },
        {
            'mass': 5.3, # kg
            'p_f': 30e6, #Pa
            't_init': -1,
            'w_border': 0, #m
#             'p_a_right': 1e5
        },
    ],
    'calc_settings': {
        'powder_settings': {
            'cell_dx': 0.0025,
            'n_cells_min': 13,
            'n_cells_max': 2000,
            'GasFluxCalculator_kwargs': {'flux_type':2},
            'GridStrecher_kwargs': {}
        },
        'gas_settings': {
            'cell_dx': 0.0025,
            'n_cells_min': 13,
            'n_cells_max': 2000,
            'GasFluxCalculator_kwargs': {'flux_type':2},
            'GridStrecher_kwargs': {}
        },
        'pist_settings': {
            'cell_dx': 0.0025,
            'n_cells_min': 13,
            'n_cells_max': 2000,
            'GasFluxCalculator_kwargs': {'flux_type':2},
            'GridStrecher_kwargs': {}
        }
    }
}


foo_tst(sandwich_dict_sample)

0.0005618510905503651

In [38]:
from distributed import Client

cl = Client('172.16.49.81:8786')
cl

Client Scheduler: tcp://172.16.49.81:8786 Dashboard: http://172.16.49.81:8787/status,Cluster Workers: 111 Cores: 122 Memory: 196.54 GB


In [24]:
from copy import deepcopy

dicts = []
for i in range(200):
    r = deepcopy(sandwich_dict_sample)
    r['i'] = i
    dicts.append(r)

futs = cl.map(foo_tst, dicts)

In [27]:
ress = cl.gather(futs)
len(ress)

KilledWorker: ('foo_tst-6f9ec743a9aa53fcc399e7c1b8a1472d', <Worker 'tcp://192.168.65.3:35239', name: tcp://192.168.65.3:35239, memory: 0, processing: 6>)

In [39]:
def foo2(i):
    import time
    time.sleep(1)
    return i

futs2 = cl.map(foo2, range(200))

In [40]:
cl.gather(futs2)

distributed.client - WARNING - Couldn't gather 187 keys, rescheduling {'foo2-136629198dcd1961c9d34618b409fd88': ('tcp://192.168.65.3:40027',), 'foo2-a1ed6c8c7af1f5a21fad0916de351ddb': ('tcp://192.168.65.3:43755',), 'foo2-0a8b5b0cc5e07bbd7dc22416be78d42c': ('tcp://192.168.65.3:38369',), 'foo2-62e77075939204aa35c38c48d60719f6': ('tcp://192.168.65.3:38171',), 'foo2-83ec6c6b73557487e9b04102c2ed91a2': ('tcp://192.168.65.3:43427',), 'foo2-bc95241113cf0a95d40c76e692d64db7': ('tcp://192.168.65.3:38369',), 'foo2-eaef7b158e55438c027a577cbffb834a': ('tcp://192.168.65.3:40531',), 'foo2-6a0704e78c5de565e5d64e5697be74cd': ('tcp://192.168.65.3:35099',), 'foo2-ff5f2eef97f0a2424fddd6a762505ba9': ('tcp://192.168.65.3:42991',), 'foo2-0d2923f96de0432538502c0281f027b4': ('tcp://192.168.65.3:37003',), 'foo2-44c97938f49254531ed92ba145289ae2': ('tcp://192.168.65.3:36043',), 'foo2-ce1a945fc76211e829ef94af5b6cb492': ('tcp://192.168.65.3:42851',), 'foo2-e1df151f733042d4abeb261b62667bbe': ('tcp://192.168.65.3:418

KilledWorker: ('foo2-976c18fc2860b06fb2adfdce62a50957', <Worker 'tcp://192.168.65.3:40547', name: tcp://192.168.65.3:40547, memory: 0, processing: 29>)

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [27]:
cl.restart()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
